In [ ]:
import blocksDB_diagramming as BDD

BDD.load_dependencies()

NAME_PREFIX="TEMP"

DBFILE,FROM_BLOCK,TO_BLOCK ="temp1.db", 54, 124
DBFILE,FROM_BLOCK,TO_BLOCK ="temp2.db", 1, 5000

imgpath="img"

In [ ]:
conn, blocknumbers = BDD.load_db_and_check_complete(DBFILE)
BDD.simple_stats(conn)
df = BDD.read_whole_table_into_dataframe(conn)
conn.close()

assert BDD.check_timestamp_format(df)

BDD.add_columns(df)
BDD.show_peak_TPS(df)
    
# if FROM_BLOCK==-1: FROM_BLOCK = min(blocknumbers)[0]
# if TO_BLOCK==-1: TO_BLOCK = max(blocknumbers)[0]

# N.B.: Redefined: FROM_BLOCK, TO_BLOCK now experiment limits!

print (FROM_BLOCK, TO_BLOCK)



##### TODO: Check whether any of these is start position dependent!


In [ ]:
df.index

In [ ]:
shift=min(df["blocknumber"])
df[FROM_BLOCK-shift:TO_BLOCK-shift+10]

In [ ]:
fn = BDD.diagrams(df, FROM_BLOCK-shift, TO_BLOCK-shift+10, NAME_PREFIX, gas_logy=True, bt_logy=True, imgpath=imgpath)

print (fn)

In [ ]:
dfs = df[FROM_BLOCK-shift:TO_BLOCK-shift+10] # just the slice of the experiment + 10 extra blocks
dfs

In [ ]:
import matplotlib.pyplot as plt
fig, axes = plt.subplots(2, 2, figsize=(16,9))


In [ ]:
def averager(dfs, col, emptyBlocks, fmt="%.1f"):
    """
    we want the real average, taken only over the non-empty blocks
    """
    av = dfs[col] [:-emptyBlocks] .mean()
    avTxt = fmt % av
    return av, avTxt

def avgLine(ax, dfs, emptyBlocks, avg, avgTxt):
    """
    horizontal line plus text on white background
    """
    blMin, blMax = min(dfs["blocknumber"]), max(dfs["blocknumber"][:-emptyBlocks])
    ax.plot([blMin, blMax], [avg, avg], "k-")
    
    ax.text(blMin + (blMax-blMin + emptyBlocks)*0.95, avg, avgTxt, 
            bbox=dict(facecolor='white', edgecolor='white'))
    
def axes_simplifier(ax, logYscale=False):
    """
    otherwise matplotlib automatically switches on notations that might be confusing
    """
    ax.get_xaxis().get_major_formatter().set_useOffset(False)
    ax.get_xaxis().get_major_formatter().set_scientific(False)
    if not logYscale:
        ax.get_yaxis().get_major_formatter().set_useOffset(False)
        ax.get_yaxis().get_major_formatter().set_scientific(False)

def tps_plotter(ax, dfs, emptyBlocks=10):
    """
    TPS average calculated only over non-empty blocks!
    average calculated for TPS (not for smoothed 3, 5, 10 blocks averages)
    """
    ax.set_title("transactions per second")
    
    cols=['TPS_1blk', 'TPS_3blks', 'TPS_5blks', 'TPS_10blks']
    for col in cols:
        ax.plot(dfs['blocknumber'], dfs[col])

    axes_simplifier(ax)
   
    avg, avgTxt = averager(dfs, cols[0], emptyBlocks, "%.1f")
    legend = [cols[0] + " (avg %s)"%avgTxt ] + cols[1:]
    ax.legend(legend); 
   
    avgLine(ax, dfs, emptyBlocks, avg, avgTxt)

  
    
fig, axes = plt.subplots(1, 1, figsize=(16,9))
# fig, axes = plt.subplots(1, 1, figsize=(6,4))
tps_plotter(axes, dfs)

In [ ]:
def btime_plotter(ax, dfs):
    # BT

    ax.set_title("blocktime since last block")
    
    ax.scatter(x=dfs['blocknumber'], y=dfs['blocktime'], c="b", marker="x")
    
    axes_simplifier(ax)

fig, axes = plt.subplots(1, 1, figsize=(16,9))
# fig, axes = plt.subplots(1, 1, figsize=(6,4))
btime_plotter(axes, dfs)    

In [ ]:
def bsize_plotter(ax, dfs, emptyBlocks):
    # blocksize
    # plus average line
    
    ax.scatter(dfs['blocknumber'], dfs['size'], c="g", marker="o")
    ax.plot(   dfs['blocknumber'], dfs['size'], "g-")
    
    avg, avgTxt = averager(dfs, 'size', emptyBlocks, "%d")
    avgLine(ax, dfs, emptyBlocks, avg, avgTxt)
    
    ax.set_title("blocksize")
    
    axes_simplifier(ax)

#fig, axes = plt.subplots(1, 1, figsize=(16,9))
fig, axes = plt.subplots(1, 1, figsize=(6,4))
bsize_plotter(axes, dfs, 10)    

In [ ]:
# just a test what happens when many many blocks

import pandas as pd
testdf = pd.DataFrame({"blocknumber" : range(100000,300000)})
testdf["size"]=40000
testdf
fig, axes = plt.subplots(1, 1, figsize=(6,4))
bsize_plotter(axes, testdf, 10)

In [ ]:
?plt.scatter

In [ ]:
?plt.plot

In [ ]:
?axes.text

In [ ]:
dfs['GLPS_1blk']

In [ ]:
def gas_plotter(ax, dfs):
    ax.set_title("gasUsed and gasLimit per second")    
    
    ax.plot(   dfs['blocknumber'], dfs['GLPS_1blk']) # , "g-")
    ax.plot(   dfs['blocknumber'], dfs['GUPS_1blk']) # 
    
    ax.set_yscale('log')
    
    #ax=df[['blocknumber', 'GLPS_1blk', 'GUPS_1blk']][blockFrom:blockTo].plot(x='blocknumber', 
    #                                                                         rot=90, ax=axes[1,1], 
    #                                                                         logy=gas_logy)
    
    axes_simplifier(ax, logYscale=True)
    ax.legend (["gasLimit/sec", "gasUsed/sec"] )

fig, axes = plt.subplots(1, 1, figsize=(6,4))
gas_plotter(axes, dfs)

In [ ]:
def diagrams(prefix, df, blockFrom, blockTo, emptyBlocks=10):
    
    
    offset=min(df["blocknumber"])
    # just the slice of the experiment + 10 extra blocks:
    df = df[FROM_BLOCK-offset:TO_BLOCK-offset+emptyBlocks+1] 
    
    # https://github.com/matplotlib/matplotlib/issues/5907#issuecomment-179001811
    import matplotlib
    matplotlib.rcParams['agg.path.chunksize'] = 10000
    

    fig, axes = plt.subplots(2, 2, figsize=(16,9))
    
    title = prefix + " blocks %d to %d" % (blockFrom, blockTo)
    fig.suptitle(title, fontsize=16)

    fig.subplots_adjust(hspace=0.25, wspace=0.20)
    
    
    tps_plotter(axes[0,0], df)
    btime_plotter(axes[0,1], df)
    bsize_plotter(axes[1,0], df, 10)  
    gas_plotter(axes[1,1], df)
    
diagrams("TEMP", df, FROM_BLOCK, TO_BLOCK)